# Elasticsearch

이 노트북에서는 Elasticsearch 벡터스토어를 시작하는 방법을 다룹니다.
 


**참고링크**

- [Elasticsearch LangChain 문서](https://python.langchain.com/docs/integrations/vectorstores/elasticsearch/)
- [LangChain 지원 VectorStore 리스트](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

In [3]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

## Elasticsearch v8.x 버전 사용

In [4]:
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings

vector_store = ElasticsearchStore(
    es_url="http://172.16.120.203:9201",
    index_name="langchain_index",
    embedding=OpenAIEmbeddings(),
    es_user="elastic",
    es_password="changeme",
)

## Elasticsearch v7.x 버전 사용


In [5]:
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai import OpenAIEmbeddings

vector_store = ElasticsearchStore(
    es_url="http://172.16.120.203:9200",
    index_name="langchain_index",
    embedding=OpenAIEmbeddings(),
    es_user="elastic",
    es_password="changeme",
    strategy=ElasticsearchStore.ExactRetrievalStrategy(),
    distance_strategy="COSINE",
)

In [6]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="오늘 아침 김치에 밥을 먹었다.",
    metadata={"source": "tweet"},
)

In [7]:
vector_store.add_documents(documents=[document_1])

['51969c3d-90a0-4f58-8a97-19d22d249f35']

In [8]:
docs = vector_store.similarity_search("아침에 뭐먹었어?")

for doc in docs:
    print(doc.page_content)

나는 아침에 김치에 밥을 먹었다.
오늘 아침 김치에 밥을 먹었다.
오늘 아침 김치에 밥을 먹었다.
오늘 아침 김치에 밥을 먹었다.


In [9]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)

# 텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("data/2025_사우회선출.txt")

# 문서 분할
split_doc1 = loader1.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1)

18

In [10]:
retriever = vector_store.as_retriever()
retriever.invoke("사우회 회장이 누구야?")

[Document(metadata={'source': 'tweet'}, page_content='나는 아침에 김치에 밥을 먹었다.'),
 Document(metadata={'source': 'tweet'}, page_content='오늘 아침 김치에 밥을 먹었다.'),
 Document(metadata={'source': 'tweet'}, page_content='오늘 아침 김치에 밥을 먹었다.'),
 Document(metadata={'source': 'tweet'}, page_content='오늘 아침 김치에 밥을 먹었다.')]